In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
# # Clear any logs from previous runs
# rm -rf ./logs/

In [3]:
from tensorboard.plugins.hparams import api as hp

In [4]:
import os

import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

from recommender import Recommender
from user_embedding import UserModel
from item_embedding import ItemModel

from datetime import datetime

In [5]:
base_loc = r'D:\dev work\recommender systems\ATRAD_CARS'

train_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/train").cache() #data\ratings_train
test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test").cache()
portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

In [6]:
model = Recommender(
    use_timestamp = True,
    portfolios = portfolios
    )

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.3))

train_ds = train_ds.shuffle(1000).batch(128) #.cache()
test_ds = test_ds.batch(128)

In [8]:
# log_dir = os.path.join(base_loc ,"logs/fit/retriever_opt/" + "retriever" + datetime.now().strftime("%Y%m%d-%H%M%S"))

In [9]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=os.path.join(base_loc ,"logs/fit/retriever_opt/" + "retriever" + datetime.now().strftime("%Y%m%d-%H%M%S")),
    histogram_freq=0,
    embeddings_freq = 1,
    write_images = True)

In [10]:
history = model.fit(
    train_ds, 
    epochs=3, 
    verbose = 1,
    validation_data=test_ds,
    validation_freq=1,
    callbacks=[tensorboard_callback]
    )

print(history.history.keys())

Epoch 1/3
25/25 [==============================] - 9s 214ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0042 - factorized_top_k/top_50_categorical_accuracy: 0.0140 - factorized_top_k/top_100_categorical_accuracy: 0.0299 - loss: 575.2949 - regularization_loss: 0.0000e+00 - total_loss: 575.2949 - val_factorized_top_k/top_1_categorical_accuracy: 0.0104 - val_factorized_top_k/top_5_categorical_accuracy: 0.0117 - val_factorized_top_k/top_10_categorical_accuracy: 0.0156 - val_factorized_top_k/top_50_categorical_accuracy: 0.0338 - val_factorized_top_k/top_100_categorical_accuracy: 0.0533 - val_loss: 0.0000e+00 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 2/3
25/25 [==============================] - 5s 182ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0019 - factorized_top_k/top_5_categorical_accuracy: 0.0029 - factorized_top_k/top_10_cate

In [11]:
res = model.evaluate(test_ds, return_dict=True)
res

7/7 [==============================] - 1s 137ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0195 - factorized_top_k/top_5_categorical_accuracy: 0.0195 - factorized_top_k/top_10_categorical_accuracy: 0.0208 - factorized_top_k/top_50_categorical_accuracy: 0.0416 - factorized_top_k/top_100_categorical_accuracy: 0.0715 - loss: 504.5273 - regularization_loss: 0.0000e+00 - total_loss: 504.5273


{'factorized_top_k/top_1_categorical_accuracy': 0.019505850970745087,
 'factorized_top_k/top_5_categorical_accuracy': 0.019505850970745087,
 'factorized_top_k/top_10_categorical_accuracy': 0.020806241780519485,
 'factorized_top_k/top_50_categorical_accuracy': 0.04161248356103897,
 'factorized_top_k/top_100_categorical_accuracy': 0.07152145355939865,
 'loss': 0.0,
 'regularization_loss': 0,
 'total_loss': 0.0}

In [12]:
res['factorized_top_k/top_10_categorical_accuracy']

0.020806241780519485

# Tuner

In [13]:
from tensorboard.plugins.hparams import api as hp

In [14]:
log_dir = os.path.join(base_loc ,"logs/retriever_opt")

In [15]:
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

hp_doc_path = os.path.join(log_dir, 'hparam_tuning')
with tf.summary.create_file_writer(hp_doc_path).as_default():
  hp.hparams_config(
    hparams=[HP_OPTIMIZER],
    metrics=[
        hp.Metric('factorized_top_k/top_10_categorical_accuracy',
        display_name='factorized_top_k/top_10_categorical_accuracy')
        ],
  )

In [16]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    top_10_categorical_accuracy = train_test_model(hparams)
    tf.summary.scalar('factorized_top_k/top_10_categorical_accuracy', top_10_categorical_accuracy, step=1)

In [17]:
def train_test_model(hparams):

  model = Recommender(
    use_timestamp = True,
    portfolios = portfolios
    )

  model.compile(
      optimizer=hparams[HP_OPTIMIZER]
  )

  # model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes

  model.fit(
    train_ds, 
    epochs=1, 
    verbose = 1,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir),  # log metrics
        hp.KerasCallback(log_dir, hparams),  # log hparams
    ],
    )
  
  res = model.evaluate(test_ds, return_dict= True)
  # print(res)
  return res['factorized_top_k/top_10_categorical_accuracy']

In [18]:
session_num = 0

for optimizer in HP_OPTIMIZER.domain.values:
    hparams = {
        HP_OPTIMIZER: optimizer,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run(os.path.join(hp_doc_path, run_name), hparams)
    session_num += 1

--- Starting trial: run-0
{'optimizer': 'adam'}
7/7 [==============================] - 2s 134ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0026 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0065 - factorized_top_k/top_50_categorical_accuracy: 0.0221 - factorized_top_k/top_100_categorical_accuracy: 0.0377 - loss: 465.5673 - regularization_loss: 0.0000e+00 - total_loss: 465.5673
--- Starting trial: run-1
{'optimizer': 'sgd'}
7/7 [==============================] - 1s 132ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0169 - factorized_top_k/top_5_categorical_accuracy: 0.0169 - factorized_top_k/top_10_categorical_accuracy: 0.0169 - factorized_top_k/top_50_categorical_accuracy: 0.0208 - factorized_top_k/top_100_categorical_accuracy: 0.0351 - loss: 465.1744 - regularization_loss: 0.0000e+00 - total_loss: 465.1744


In [24]:
# #\hparam_tuning
# %tensorboard --logdir logs\fit